In [1]:
!pip install rasterio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 7.7 MB/s eta 0:00:00


In [2]:
import io
import ee
from google.colab import auth
import geemap
import ipywidgets as widgets
from IPython.display import display, clear_output
from google.oauth2 import service_account
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
import os
import re
import geemap
import ipywidgets as widgets
import googleapiclient
from googleapiclient.discovery import build
from google.oauth2 import service_account
import geopandas as gpd
from osgeo import gdal, ogr, osr
import shutil
import rasterio
import joblib
import numpy as np
import pandas as pd
# Autentica e inizializza GEE
ee.Authenticate()
ee.Initialize(project='ee-andreagonnelli06')

In [3]:
# This cell and the next one are necessary just when we are using collab. otherwise, we need to put the address of the pre-trained model.
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
!cp /content/drive/MyDrive/rf.pkl /content/

cp: cannot stat '/content/drive/MyDrive/rf.pkl': No such file or directory


In [7]:

Map = geemap.Map()
directory_path = '/content/Gdw'  # Replace with your desired directory path

# Create the directory if it does not exist
os.makedirs(directory_path, exist_ok=True)

coordinates = {
    "coordinates":
    [
      [
        [
          11.701965902697452,
          43.64560837796924
        ],
        [
          11.704162758068293,
          43.63902803222882
        ],
        [
          11.72024021486149,
          43.64114007011807
        ],
        [
          11.714777770570237,
          43.647780227105414
        ],
        [
          11.709285235937088,
          43.64434756218202
        ],
        [
          11.701965902697452,
          43.64560837796924
        ]
      ]
    ]
}

rectangles = ee.Geometry.Polygon(coordinates["coordinates"])
Map.centerObject(rectangles)

# Function to filter and merge collections based on dates
def filter_and_merge_collections(start_date, end_date):
    image = geemap.dynamic_world_s2(rectangles, start_date, end_date)\
    .reproject(crs='EPSG:4326', scale=10)
    landcover = geemap.dynamic_world(rectangles, start_date, end_date, return_type="hillshade")
    landcover = geemap.dynamic_world(rectangles, start_date, end_date, return_type="class")
    combined_image = landcover
    display(combined_image)
    return combined_image

# Function to create monthly composites
def create_monthly_composites(start_year, end_year):
    monthly_composites = []
    for year in range(start_year, end_year + 1):
        for month in range(1, 13):
            start_date = ee.Date.fromYMD(year, month, 1)
            end_date = start_date.advance(1, 'month')
            combined_image = filter_and_merge_collections(start_date, end_date)
            monthly_composites.append(combined_image.set('year', year).set('month', month))
    return ee.ImageCollection.fromImages(monthly_composites)

# Function to visualize the monthly composites
def on_generate_button_clicked():
    start_year = 2024
    end_year = 2024
    global monthly_composites
    monthly_composites = create_monthly_composites(start_year, end_year)
    # Create a map
    Map = geemap.Map()
    # Center the map on the area of interest
    Map.centerObject(rectangles, zoom=8)
    # Add the monthly composite layers to the map
    for year in range(start_year, end_year + 1):
        for month in range(1, 13):
            composite = monthly_composites.filter(ee.Filter.eq('year', year)).filter(ee.Filter.eq('month', month)).first()
            layer_name = f'{year}-{month:02d} Composite'
            Map.addLayer(composite, {}, layer_name)
    Map.addLayer(rectangles, {}, 'Area of Interest')
    display(Map)

# Function to generate smaller bounding boxes within a larger region
def generate_tiles(region, tile_size):
    region = ee.Geometry(region)
    coords = region.bounds().getInfo()['coordinates'][0]
    min_lon, min_lat = coords[0]
    max_lon, max_lat = coords[2]
    tiles = []
    lat = min_lat
    while lat < max_lat:
        lon = min_lon
        while lon < max_lon:
            tile = ee.Geometry.Rectangle([lon, lat, min(lon + tile_size, max_lon), min(lat + tile_size, max_lat)])
            tiles.append(tile)
            lon += tile_size
        lat += tile_size
    return tiles

# Function to download the last available image
def on_download_button_clicked():
    start_year = 2024
    start_month = 3
    end_year = 2024
    end_month = 6
    out_dir = 'Gdw'  # Local directory
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    # Define the approximate size of each tile in degrees
    tile_size = 0.22  # This value can be adjusted
    for year in range(start_year, end_year + 1):
        for month in range(start_month, end_month + 1):
            # Get the last available image for the given month
            last_available_image = monthly_composites.filter(ee.Filter.eq('year', year)).filter(ee.Filter.eq('month', month)).sort('system:time_start', False).first()
            file_name_prefix = f'{year}_{month:02d}'
            tiles = generate_tiles(rectangles, tile_size)  # Generate tiles
            for i, tile in enumerate(tiles):
                file_name = f'{file_name_prefix}_tile_{i:02d}.tif'
                file_path = os.path.join(out_dir, file_name)
                try:
                    geemap.ee_export_image(last_available_image, filename=file_path, scale=10, region=tile)
                    print(f'{file_name} downloaded successfully.')
                except Exception as e:
                    print(f'Error downloading {file_name}: {e}')

# Execute the function to generate and visualize the images
on_generate_button_clicked()

# Execute the function to download the images
on_download_button_clicked()


Map(center=[43.64280994391492, 11.710766681953112], controls=(WidgetControl(options=['position', 'transparent_…

Generating URL ...
Please wait ...
Data downloaded to /content/Gdw/2024_03_tile_00.tif
2024_03_tile_00.tif downloaded successfully.
Generating URL ...
Please wait ...
Data downloaded to /content/Gdw/2024_04_tile_00.tif
2024_04_tile_00.tif downloaded successfully.
Generating URL ...
Please wait ...
Data downloaded to /content/Gdw/2024_05_tile_00.tif
2024_05_tile_00.tif downloaded successfully.
Generating URL ...
Please wait ...
Data downloaded to /content/Gdw/2024_06_tile_00.tif
2024_06_tile_00.tif downloaded successfully.


In [9]:
import os
import json
from osgeo import gdal, ogr, osr
import numpy as np

VIS_PALETTE = [
    '419bdf', '397d49', '88b053', '7a87c6', 'e49635', 'dfc35a', 'c4281b',
    'a59b8f', 'b39fe1'
]

FIELD_NAMES = [
    'water', 'trees', 'grass', 'flooded_vegetation',
    'crops', 'shrub_and_scrub', 'built', 'bare', 'snow_and_ice'
]

def read_band_as_array(raster_path, band_index):
    src_ds = gdal.Open(raster_path)
    if src_ds is None:
        print(f"Unable to open {raster_path}")
        return None

    if band_index < 1 or band_index > src_ds.RasterCount:
        print(f"Band index {band_index} is out of range for {raster_path}")
        return None

    band = src_ds.GetRasterBand(band_index)
    arr = band.ReadAsArray()

    src_ds = None  # Close the dataset

    return arr

def create_pixel_polygons(image_path, field_names, colors):
    src_ds = gdal.Open(image_path)
    if src_ds is None:
        print(f"Unable to open {image_path}")
        return []

    transform = src_ds.GetGeoTransform()
    rows, cols = src_ds.RasterYSize, src_ds.RasterXSize

    # Assuming the class labels are in the first band
    class_labels = read_band_as_array(image_path, 1)
    if class_labels is None:
        print("No valid band found in the image")
        return []

    srs = osr.SpatialReference()
    srs.ImportFromWkt(src_ds.GetProjection())
    srs.SetWellKnownGeogCS("EPSG:4326")

    features = []
    for i in range(rows):
        for j in range(cols):
            class_index = int(class_labels[i, j])

            if class_index < 0 or class_index >= len(field_names):
                print(f"Invalid class index {class_index} at pixel ({i}, {j})")
                continue

            field_name = field_names[class_index]
            color = colors[class_index]

            minx = transform[0] + j * transform[1]
            maxx = transform[0] + (j + 1) * transform[1]
            miny = transform[3] + i * transform[5]
            maxy = transform[3] + (i + 1) * transform[5]

            ring = ogr.Geometry(ogr.wkbLinearRing)
            ring.AddPoint(minx, miny)
            ring.AddPoint(maxx, miny)
            ring.AddPoint(maxx, maxy)
            ring.AddPoint(minx, maxy)
            ring.AddPoint(minx, miny)

            poly = ogr.Geometry(ogr.wkbPolygon)
            poly.AddGeometry(ring)

            poly.TransformTo(srs)

            feature = {
                "type": "Feature",
                "geometry": json.loads(poly.ExportToJson()),
                "properties": {
                    "class": field_name,
                    "value": class_index,
                    'color': f'#{color}'
                }
            }
            features.append(feature)

    return features

def process_images_in_folder(folder_path, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for i, image_path in enumerate(os.listdir(folder_path)):
        if image_path.endswith('.tif'):
            full_image_path = os.path.join(folder_path, image_path)
            features = create_pixel_polygons(full_image_path, FIELD_NAMES, VIS_PALETTE)

            geojson = {
                "type": "FeatureCollection",
                "features": features
            }

            file_name = f'combined_tile_{i:02d}.geojson'
            output_geojson_path = os.path.join(output_folder, file_name)

            # Write the GeoJSON
            with open(output_geojson_path, 'w') as f:
                f.write(json.dumps(geojson, indent=2))

            print(f"GeoJSON created: {output_geojson_path}")

# Example usage
image_folder = 'Gdw'
output_geojson_folder = 'output_geojsons'
process_images_in_folder(image_folder, output_geojson_folder)


GeoJSON created: output_geojsons/combined_tile_00.geojson
GeoJSON created: output_geojsons/combined_tile_01.geojson
GeoJSON created: output_geojsons/combined_tile_02.geojson
GeoJSON created: output_geojsons/combined_tile_04.geojson


In [11]:
import os
import json
from osgeo import gdal, ogr, osr
import numpy as np

VIS_PALETTE = [
    '419bdf', '397d49', '88b053', '7a87c6', 'e49635', 'dfc35a', 'c4281b',
    'a59b8f', 'b39fe1'
]

FIELD_NAMES = [
    'water', 'trees', 'grass', 'flooded_vegetation',
    'crops', 'shrub_and_scrub', 'built', 'bare', 'snow_and_ice'
]

def read_band_as_array(raster_path, band_index):
    src_ds = gdal.Open(raster_path)
    if src_ds is None:
        print(f"Unable to open {raster_path}")
        return None

    if band_index < 1 or band_index > src_ds.RasterCount:
        print(f"Band index {band_index} is out of range for {raster_path}")
        return None

    band = src_ds.GetRasterBand(band_index)
    arr = band.ReadAsArray()

    src_ds = None  # Close the dataset

    return arr

def create_pixel_polygons(image_path, field_names, colors):
    src_ds = gdal.Open(image_path)
    if src_ds is None:
        print(f"Unable to open {image_path}")
        return []

    transform = src_ds.GetGeoTransform()
    rows, cols = src_ds.RasterYSize, src_ds.RasterXSize

    # Assuming the class labels are in the first band
    class_labels = read_band_as_array(image_path, 1)
    if class_labels is None:
        print("No valid band found in the image")
        return []

    srs = osr.SpatialReference()
    srs.ImportFromWkt(src_ds.GetProjection())
    srs.SetWellKnownGeogCS("EPSG:4326")

    features = []
    for i in range(rows):
        for j in range(cols):
            class_index = int(class_labels[i, j])

            if class_index < 0 or class_index >= len(field_names):
                print(f"Invalid class index {class_index} at pixel ({i}, {j})")
                continue

            field_name = field_names[class_index]
            color = colors[class_index]

            minx = transform[0] + j * transform[1]
            maxx = transform[0] + (j + 1) * transform[1]
            miny = transform[3] + i * transform[5]
            maxy = transform[3] + (i + 1) * transform[5]

            ring = ogr.Geometry(ogr.wkbLinearRing)
            ring.AddPoint(minx, miny)
            ring.AddPoint(maxx, miny)
            ring.AddPoint(maxx, maxy)
            ring.AddPoint(minx, maxy)
            ring.AddPoint(minx, miny)

            poly = ogr.Geometry(ogr.wkbPolygon)
            poly.AddGeometry(ring)

            poly.TransformTo(srs)

            feature = {
                "type": "Feature",
                "geometry": json.loads(poly.ExportToJson()),
                "properties": {
                    "class": field_name,
                    "value": class_index,
                    'color': f'#{color}'
                }
            }
            features.append(feature)

    return features

def process_images_in_folder(folder_path, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for image_path in os.listdir(folder_path):
        if image_path.endswith('.tif'):
            full_image_path = os.path.join(folder_path, image_path)
            features = create_pixel_polygons(full_image_path, FIELD_NAMES, VIS_PALETTE)

            geojson = {
                "type": "FeatureCollection",
                "features": features
            }

            file_name = os.path.splitext(image_path)[0] + '.geojson'
            output_geojson_path = os.path.join(output_folder, file_name)

            # Write the GeoJSON
            with open(output_geojson_path, 'w') as f:
                f.write(json.dumps(geojson, indent=2))

            print(f"GeoJSON created: {output_geojson_path}")

# Example usage
image_folder = 'Gdw'
output_geojson_folder = 'output_geojsons'
process_images_in_folder(image_folder, output_geojson_folder)


GeoJSON created: output_geojsons/2024_05_tile_00.geojson
GeoJSON created: output_geojsons/2024_03_tile_00.geojson
GeoJSON created: output_geojsons/2024_04_tile_00.geojson
GeoJSON created: output_geojsons/2024_06_tile_00.geojson
